In [ ]:
# Install necessary libraries

!pip3 install pandas 
!pip3 install plotly.express
!pip3 install streamlit 
import pandas as pd 
import plotly.express as px
import streamlit as st


In [ ]:
# Import IMF Data from google sheets 

import pandas as pd
url = "https://docs.google.com/spreadsheets/d/1BV0koOEqQs580tEPGv9bpZYUfY8q8UTfZGTcEoK_VtQ/export?format=csv"
df = pd.read_csv(url)

/var/folders/y5/yvs2_16n2ns9rdh4zn1ffl400000gn/T/ipykernel_75154/2769739119.py:6: DtypeWarning: Columns (9,14,15,17,51,53,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


In [47]:
# Built table for analytics including indicators and countries 

# Select columns used for analysis, e.g. years 1980-2024, country code, frequency etc.  

year_cols = [str(y) for y in range(1980, 2025)] # yearly columns

fixed_cols = ['SERIES_CODE', 'COUNTRY', 'INDICATOR', 'FREQUENCY', 
              'SCALE', 'PRICES', 'UNIT', 'TOPIC'] # non-yearly columns

df_selected = df[fixed_cols + year_cols]

# Adjust column width

pd.set_option('display.max_colwidth', None)



In [ ]:
# Select specific countries for analysis

df_countries = df_selected[df_selected['COUNTRY'].isin(['Germany', 'Denmark', 'Poland','United States','Chile','Costa Rica','Japan','South Africa', 'Ghana','Ivory Coast'])] 

df_countries

# Select specific indicators: GDP per capita, Unemployment, Consumer Price Index (Inflation), Savings, Investment  

df_countries_analysis = df_countries[df_countries['INDICATOR'].isin(['Gross domestic product (GDP) per capita, current prices, U.S. dollars','Gross national savings, Percent of GDP','All Items, Consumer price index (CPI), End-of-period (EoP)','Unemployment rate'])]


In [96]:
# Rename Indicators for better visualization

indicator_map = {
    'Gross domestic product (GDP) per capita, current prices, U.S. dollars': 'GDP per capita (adjusted for purchasing power parity)',
    'Gross national savings, Percent of GDP': 'National savings (% GDP)',
    'All Items, Consumer price index (CPI), End-of-period (EoP)': 'Inflation (CPI, %))',
    'Unemployment rate': 'Unemployment levels (%)'
}

df_countries_analysis['INDICATOR'] = df_countries_analysis['INDICATOR'].map(indicator_map)

df_countries_analysis


/var/folders/y5/yvs2_16n2ns9rdh4zn1ffl400000gn/T/ipykernel_75154/825008160.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_countries_analysis['INDICATOR'] = df_countries_analysis['INDICATOR'].map(indicator_map)


,SERIES_CODE,COUNTRY,INDICATOR,FREQUENCY,SCALE,PRICES,UNIT,TOPIC,1980,1981,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
152,USA.NGSD_NGDP.A,United States,NaN,Annual,Units,NaN,Percent,Gross domestic product (GDP),22.059,23.206,...,19.611,18.472,18.882,19.126,19.329,18.232,17.583,18.186,17.026,16.494
156,USA.LUR.A,United States,NaN,Annual,Units,NaN,NaN,Unemployment,7.175,7.617,...,5.275,4.875,4.358,3.892,3.675,8.100,5.358,3.642,3.633,4.033
237,USA.PCPIE.A,United States,NaN,Annual,Units,NaN,Index,Consumer prices,86.750,94.183,...,237.763,242.987,248.280,253.048,258.353,262.389,281.782,299.845,309.566,318.015
273,DEU.LUR.A,Germany,NaN,Annual,Units,NaN,NaN,Unemployment,3.359,4.831,...,4.358,3.900,3.525,3.192,2.950,3.625,3.600,3.142,3.083,3.392
340,DEU.PCPIE.A,Germany,NaN,Annual,Units,NaN,Index,Consumer prices,NaN,NaN,...,99.600,101.300,102.900,104.800,106.233,105.600,111.367,123.400,127.067,130.267
424,DNK.PCPIE.A,Denmark,NaN,Annual,Units,NaN,Index,Consumer prices,37.754,42.435,...,100.100,100.400,101.300,102.100,103.000,103.500,107.100,117.500,118.000,120.200
434,DNK.LUR.A,Denmark,NaN,Annual,Units,NaN,NaN,Unemployment,5.177,6.980,...,4.542,4.125,4.200,3.850,3.658,4.658,3.592,2.508,2.767,2.900
453,DNK.NGSD_NGDP.A,Denmark,NaN,Annual,Units,NaN,Percent,Gross domestic product (GDP),19.588,16.357,...,28.500,28.768,29.280,28.840,29.332,29.989,32.507,35.940,34.650,35.400
467,DEU.NGSD_NGDP.A,Germany,NaN,Annual,Units,NaN,Percent,Gross domestic product (GDP),21.462,20.081,...,27.599,28.621,28.753,30.072,29.254,28.047,29.582,27.087,27.512,27.131
911,JPN.PCPIE.A,Japan,NaN,Annual,Units,NaN,Index,Consumer prices,74.783,77.807,...,98.186,98.445,98.983,99.841,100.351,99.430,99.931,103.788,106.807,109.925


In [124]:
# replace NaN values in Unit Column with percent if the indicator is Unemployment
df_countries_analysis.loc[df_countries_analysis['INDICATOR'] == 'Unemployment levels (%)', 'UNIT'] = 'Percent'

df_countries_analysis


,SERIES_CODE,COUNTRY,INDICATOR,FREQUENCY,SCALE,PRICES,UNIT,TOPIC,1980,1981,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
152,USA.NGSD_NGDP.A,United States,NaN,Annual,Units,NaN,Percent,Gross domestic product (GDP),22.059,23.206,...,19.611,18.472,18.882,19.126,19.329,18.232,17.583,18.186,17.026,16.494
156,USA.LUR.A,United States,NaN,Annual,Units,NaN,NaN,Unemployment,7.175,7.617,...,5.275,4.875,4.358,3.892,3.675,8.100,5.358,3.642,3.633,4.033
237,USA.PCPIE.A,United States,NaN,Annual,Units,NaN,Index,Consumer prices,86.750,94.183,...,237.763,242.987,248.280,253.048,258.353,262.389,281.782,299.845,309.566,318.015
273,DEU.LUR.A,Germany,NaN,Annual,Units,NaN,NaN,Unemployment,3.359,4.831,...,4.358,3.900,3.525,3.192,2.950,3.625,3.600,3.142,3.083,3.392
340,DEU.PCPIE.A,Germany,NaN,Annual,Units,NaN,Index,Consumer prices,NaN,NaN,...,99.600,101.300,102.900,104.800,106.233,105.600,111.367,123.400,127.067,130.267
424,DNK.PCPIE.A,Denmark,NaN,Annual,Units,NaN,Index,Consumer prices,37.754,42.435,...,100.100,100.400,101.300,102.100,103.000,103.500,107.100,117.500,118.000,120.200
434,DNK.LUR.A,Denmark,NaN,Annual,Units,NaN,NaN,Unemployment,5.177,6.980,...,4.542,4.125,4.200,3.850,3.658,4.658,3.592,2.508,2.767,2.900
453,DNK.NGSD_NGDP.A,Denmark,NaN,Annual,Units,NaN,Percent,Gross domestic product (GDP),19.588,16.357,...,28.500,28.768,29.280,28.840,29.332,29.989,32.507,35.940,34.650,35.400
467,DEU.NGSD_NGDP.A,Germany,NaN,Annual,Units,NaN,Percent,Gross domestic product (GDP),21.462,20.081,...,27.599,28.621,28.753,30.072,29.254,28.047,29.582,27.087,27.512,27.131
911,JPN.PCPIE.A,Japan,NaN,Annual,Units,NaN,Index,Consumer prices,74.783,77.807,...,98.186,98.445,98.983,99.841,100.351,99.430,99.931,103.788,106.807,109.925


In [87]:
columns_tokeep=['COUNTRY','TOPIC']

columns_tomelt=[str(y) for y in range(1980, 2025)]

In [90]:
# Visualization of GDP per capita over time;

df_countries_analysis

df_melted =pd.melt(df_countries_analysis, id_vars=columns_tokeep, value_vars=columns_tomelt, var_name='Year', value_name='Value')

In [92]:
df_melted.sort_values(by=['COUNTRY', 'Year'], inplace=True)

In [95]:
df_melted.head(45)

,COUNTRY,TOPIC,Year,Value
16,Chile,Unemployment,1980,11.502
19,Chile,Consumer prices,1980,4.039
20,Chile,Gross domestic product (GDP),1980,12.366
39,Chile,Unemployment,1981,10.291
42,Chile,Consumer prices,1981,4.490
43,Chile,Gross domestic product (GDP),1981,7.870
62,Chile,Unemployment,1982,19.814
65,Chile,Consumer prices,1982,5.419
66,Chile,Gross domestic product (GDP),1982,1.467
85,Chile,Unemployment,1983,20.999


SyntaxError: unexpected EOF while parsing (3263373558.py, line 1)